# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [22]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import holoviews as hv



# Import API key
from api_keys import geoapify_key

In [23]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,20.31,90,0,2.06,US,1697993594
1,1,albany,42.6001,-73.9662,8.86,76,100,0.45,US,1697993453
2,2,blackmans bay,-43.0167,147.3167,11.92,88,86,1.79,AU,1697993594
3,3,margaret river,-33.9500,115.0667,13.37,59,93,7.27,AU,1697993595
4,4,el granada,37.5027,-122.4694,17.53,89,100,4.12,US,1697993314


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [35]:
%%capture --no-display

import geoviews as gv
import geoviews.tile_sources as gts

# Configure the map plot
map_plot = gv.Points(city_data_df, kdims=['Lng', 'Lat'], vdims=['Humidity', 'City'])
map_plot = map_plot.opts(
    color='Humidity', size=gv.dim('Humidity') * 0.1, cmap='viridis', colorbar=True,
    width=500, height=300, tools=['hover'], title='City Map with Humidity'
)

# Defining the base map as geographical
base_map = gts.CartoLight.opts(width=800, height=600, xaxis=None, yaxis=None)

# Overlaying the data from the scatterplot onto the map
geographical_map = base_map * map_plot

# Display the map
display(geographical_map)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = 27  # Max temperature lower than 27 degrees
min_temp = 21  # Min temperature higher than 21 degrees
max_wind_speed = 4.5  # Wind speed less than 4.5 m/s
max_cloudiness = 0  # Zero cloudiness

# filter data
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] < max_temp) &
    (city_data_df['Max Temp'] > min_temp) &
    (city_data_df['Wind Speed'] < max_wind_speed) &
    (city_data_df['Cloudiness'] == max_cloudiness)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
89,89,abu al matamir,30.9102,30.1744,22.09,76,0,4.21,EG,1697993620
108,108,tsiombe,-25.3000,45.4833,24.15,67,0,2.72,MG,1697993626
110,110,modimolle,-24.7000,28.4000,25.83,8,0,3.57,ZA,1697993627
149,149,pecos,31.4229,-103.4932,22.06,46,0,0.00,US,1697993639
161,161,pass christian,30.3158,-89.2475,26.61,57,0,0.00,US,1697993642


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
89,89,abu al matamir,30.9102,30.1744,22.09,76,0,4.21,EG,1697993620,
108,108,tsiombe,-25.3000,45.4833,24.15,67,0,2.72,MG,1697993626,
110,110,modimolle,-24.7000,28.4000,25.83,8,0,3.57,ZA,1697993627,
149,149,pecos,31.4229,-103.4932,22.06,46,0,0.00,US,1697993639,
161,161,pass christian,30.3158,-89.2475,26.61,57,0,0.00,US,1697993642,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add bias parameter for proximity to the current city's latitude and longitude
    params = {
        "radius": radius,
        "categories": "accommodation.hotel",  # Updated to use specific category
        "bias": f"proximity:{longitude},{latitude}",  # Bias by proximity
        "apiKey": geoapify_key
    }

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Check the HTTP status code
    if response.status_code == 200:
        print(f"API request for {hotel_df.loc[index, 'City']} successful. Connected to the database.")
        name_address = response.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"

        # Log the search results
        print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
API request for abu al matamir successful. Connected to the database.
abu al matamir - nearest hotel: نادي لاجون
API request for tsiombe successful. Connected to the database.
tsiombe - nearest hotel: Le Cactus
API request for modimolle successful. Connected to the database.
modimolle - nearest hotel: Mezi Mezi Guest House
API request for pecos successful. Connected to the database.
pecos - nearest hotel: Holiday Inn Express & Suites Pecos
API request for pass christian successful. Connected to the database.
pass christian - nearest hotel: Pearl Hotel
API request for bronkhorstspruit successful. Connected to the database.
bronkhorstspruit - nearest hotel: No hotel found
API request for sainte-marie successful. Connected to the database.
sainte-marie - nearest hotel: Hôtel Select Océan Indien
API request for christiana successful. Connected to the database.
christiana - nearest hotel: Christiana All Seasons Hotel & Conference Centre
API request for cannonvale succe

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
89,89,abu al matamir,30.9102,30.1744,22.09,76,0,4.21,EG,1697993620,نادي لاجون
108,108,tsiombe,-25.3000,45.4833,24.15,67,0,2.72,MG,1697993626,Le Cactus
110,110,modimolle,-24.7000,28.4000,25.83,8,0,3.57,ZA,1697993627,Mezi Mezi Guest House
149,149,pecos,31.4229,-103.4932,22.06,46,0,0.00,US,1697993639,Holiday Inn Express & Suites Pecos
161,161,pass christian,30.3158,-89.2475,26.61,57,0,0.00,US,1697993642,Pearl Hotel
162,162,bronkhorstspruit,-25.8102,28.7425,22.18,29,0,3.18,ZA,1697993643,No hotel found
207,207,sainte-marie,-20.8968,55.5496,22.62,87,0,2.57,RE,1697993657,Hôtel Select Océan Indien
215,215,christiana,-27.9140,25.1611,24.98,17,0,3.47,ZA,1697993660,Christiana All Seasons Hotel & Conference Centre
258,258,cannonvale,-20.2833,148.7000,21.66,75,0,1.74,AU,1697993672,Whitsundays Rainforest Retreat
458,458,faisalabad,31.4167,73.0833,24.78,45,0,1.89,PK,1697993734,New Lazzat Kada Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display


# Configure the map plot
tiles = gv.tile_sources.CartoLight.opts(width=800, height=600, xaxis=None, yaxis=None)
points = gv.Points(hotel_df, kdims=['Lng', 'Lat'], vdims=['Humidity', 'City', 'Hotel Name', 'Country'])
points_plot = points.opts(
    color='Humidity', size=gv.dim('Humidity') * 0.27, cmap='viridis', 
    colorbar=True, 
    width=700, height=500, tools=['hover'],
    title='City Map with Hotels',  # Set the title for the plot
    ylabel='Humidity'  # Set the ylabel as 'Humidity'
)

#overlay points
geographical_map = tiles * points_plot


# Display the map
display(geographical_map)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)